In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

try:
    conn = mysql.connector.connect(host='localhost',
                                   database='sports_db',
                                   user='root',
                                   password='Krish_1305')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

        # Create Dimension and Fact Tables
        cursor.execute('DROP TABLE IF EXISTS fact_player_performance;')
        cursor.execute('DROP TABLE IF EXISTS dim_player;')
        cursor.execute('DROP TABLE IF EXISTS dim_team;')
        cursor.execute('DROP TABLE IF EXISTS dim_season;')

        cursor.execute("""
            CREATE TABLE dim_player (
                player_id INT AUTO_INCREMENT PRIMARY KEY,
                player_name VARCHAR(255),
                age FLOAT,
                height FLOAT,
                weight FLOAT,
                position VARCHAR(255),
                injury_history FLOAT,
                last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
            );
        """)

        cursor.execute("""
            CREATE TABLE dim_team (
                team_id INT AUTO_INCREMENT PRIMARY KEY,
                team_name VARCHAR(255),
                last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
            );
        """)

        cursor.execute("""
            CREATE TABLE dim_season (
                season_id INT AUTO_INCREMENT PRIMARY KEY,
                season_year FLOAT,
                last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
            );
        """)

        cursor.execute("""
            CREATE TABLE fact_player_performance (
                id INT AUTO_INCREMENT PRIMARY KEY,
                player_id INT,
                team_id INT,
                season_id INT,
                goals FLOAT,
                assists FLOAT,
                yellow_cards FLOAT,
                red_cards FLOAT,
                pass_completion_rate FLOAT,
                distance_covered FLOAT,
                sprints FLOAT,
                shots_on_target FLOAT,
                tackles_won FLOAT,
                clean_sheets FLOAT,
                player_fatigue FLOAT,
                match_pressure FLOAT,
                training_hours FLOAT,
                fatigue_injury_correlation FLOAT,
                pressure_performance_impact FLOAT,
                effective_training FLOAT,
                FOREIGN KEY (player_id) REFERENCES dim_player(player_id),
                FOREIGN KEY (team_id) REFERENCES dim_team(team_id),
                FOREIGN KEY (season_id) REFERENCES dim_season(season_id),
                last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
            );
        """)

        print("Tables created successfully")
        
        # Read data from CSV
        file_path = 'Sports_Dataset_Cleaned.csv'
        sports_data = pd.read_csv(file_path)

        # Inspect the data
        print(sports_data.head())
        print(sports_data.columns)

        # Adjust these columns based on your actual CSV column names
        players_columns = ['player', 'age', 'height', 'weight', 'position', 'injuryhistory']
        teams_columns = ['team']
        seasons_columns = ['season']

        # Insert data into dim_player
        players = sports_data[players_columns].drop_duplicates().reset_index(drop=True)
        for _, row in players.iterrows():
            cursor.execute("""
                INSERT INTO dim_player (player_name, age, height, weight, position, injury_history) 
                VALUES (%s, %s, %s, %s, %s, %s);
            """, tuple(row))
        conn.commit()
        print("inserted of dim_player")

        # Insert data into dim_team
        teams = sports_data[teams_columns].drop_duplicates().reset_index(drop=True)
        for _, row in teams.iterrows():
            cursor.execute("""
                INSERT INTO dim_team (team_name) 
                VALUES (%s);
            """, tuple(row))
        conn.commit()
        print("inserted of dim_team")

        # Insert data into dim_season
        seasons = sports_data[seasons_columns].drop_duplicates().reset_index(drop=True)
        for _, row in seasons.iterrows():
            cursor.execute("""
                INSERT INTO dim_season (season_year) 
                VALUES (%s);
            """, tuple(row))
        conn.commit()
        print("inserted of dim_season")

        # Insert data into fact_player_performance
        for _, row in sports_data.iterrows():
            cursor.execute("""
                INSERT INTO fact_player_performance (player_id, team_id, season_id, goals, assists, yellow_cards, red_cards, pass_completion_rate, distance_covered, sprints, shots_on_target, tackles_won, clean_sheets, player_fatigue, match_pressure, training_hours, fatigue_injury_correlation, pressure_performance_impact, effective_training)
                SELECT p.player_id, t.team_id, s.season_id, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                FROM dim_player p, dim_team t, dim_season s
                WHERE p.player_name = %s AND t.team_name = %s AND s.season_year = %s;
            """, (
                row['goals'], row['assists'], row['yellowcards'], row['redcards'], row['passcompletionrate'], row['distancecovered'], row['sprints'], row['shotsontarget'], row['tackleswon'], row['cleansheets'], row['playerfatigue'], row['matchpressure'], row['traininghours'], row['fatigueinjurycorrelation'], row['pressureperformanceimpact'], row['effectivetraining'], 
                row['player'], row['team'], row['season']
            ))
        conn.commit()

        print("Data inserted successfully")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection is closed")


You're connected to database:  ('sports_db',)
Tables created successfully
     player    team  age      height     weight  position  goals  assists  \
0  Player C  Team C   31  164.238236  64.899554  Defender   11.0      2.0   
1  Player D  Team C   22  164.489643  55.636159  Defender    2.0     16.0   
2  Player C  Team C   29  184.567349  50.952308   Forward   13.0      1.0   
3  Player C  Team C   27  192.172813  78.832881  Defender    6.0      8.0   
4  Player D  Team A   28  195.970012  55.136881   Forward    6.0      3.2   

   yellowcards  redcards  ...  tackleswon  cleansheets  playerfatigue  \
0            4         3  ...          22            6       0.374540   
1            8         2  ...          27            6       0.950714   
2            3         3  ...           3            4       0.598658   
3            1         2  ...          12            8       0.156019   
4            1         4  ...          23            1       0.155995   

   matchpressure  injury